Klasse matcher_localizer

Findet Patterns in einer Anleitung mit einem einfachen spacy PhraseMatcher. Zuordnung anhand von Wortlisten (z.B. aus der REST Schnittstelle).


Funktionen: 

    init_matcher(language, pattern_terms)
        language:       Sprache als String z.B.: "de"
        pattern_terms:  Dictionary mit Key: Pattern ID; Value: Liste mit Worten zu dem Pattern. get_all_pattern_evidence_words aus der REST Schnittstelle liefert ein passendes Dictionary
    
    read_nlp_processed_game_to_game_patterns(nlp_processed_game)
    nlp_processed_game:      nlp_processed_game Objekt
    returns:                 game_patterns Objekt

**Nicht verwenden!** <br>
Der matcher_localizer ist veraltet und sollte nicht länger verwendet werden. Er wurde durch den regex_localizer ersetzt.

In [1]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
#import nltk
#from nltk import tokenize
#nltk.download('punkt')

In [2]:
import spacy
from spacy.matcher import PhraseMatcher
%run ./GamePatterns.ipynb
%run ./nlpProcessedGame.ipynb

In [ ]:
class matcher_localizer:
    #def __init__ (self, *args, **kwargs):
        
        
    def init_matcher(self,spacy_nlp, pattern_terms):
        self.nlp = spacy_nlp
        self.pattern_terms = pattern_terms
        self.min_length = 0
        self.matcher = PhraseMatcher(self.nlp.vocab, attr="LOWER")
        #alle Patterns in den Matcher laden
        for patternID, terms in pattern_terms.items():
            patterns = [self.nlp.make_doc(text) for text in terms]
            self.matcher.add(str(patternID),None, *patterns)
            
    def init_matcher_lemma(self,spacy_nlp, pattern_terms):
        self.nlp = spacy_nlp
        self.pattern_terms = pattern_terms
        self.min_length = 0
        self.matcher = PhraseMatcher(self.nlp.vocab, attr="LEMMA")
        #alle Patterns in den Matcher laden
        for patternID, terms in pattern_terms.items():
            patterns = [self.nlp(text) for text in terms]
            self.matcher.add(str(patternID),None, *patterns)
            
    def set_min_sentence_length(self, min_length):
        self.min_length = min_length
        
    def read_nlp_processed_game_to_game_patterns(self,nlp_processed_game):
        gp = game_patterns()
        gp.set_ID(nlp_processed_game.get_ID())
        doc=nlp_processed_game.get_spacy_doc()
        #Alle Satzanfänge ermitteln
        sentenceStarts = []
        sentenceTexts = []
        for sent in doc.sents:
            sentenceStarts.append(sent.start)
            sentenceTexts.append(str(sent))
        gp.set_sentences(sentenceTexts)
        #print("Starts:")
        #print(sentenceStarts)
        sentences = {}
        for key, value in self.pattern_terms.items():
            sentences[str(key)] = []
        matches = self.matcher(doc)
        currSent = 0
        currSentLength = 0
        #Alle Sätze durchlaufen und Prüfen, ob das aktuelle Match in diesem Satz vorkommt
        #Dabei wird ausgenutzt, dass die Matches sortiert zurück gegeben werden.
        #So müssen die Sätze nur ein einziges mal durchlaufen werden.
        for match_id, start, end in matches:
            #print("ID: "+ nlp.vocab.strings[match_id] + " start: " + str(start) + " end: "+ str(end)+ " pattern found: " + doc[start:end].text)
            
            #Eins weiter bis der Satz mit dem Match gefunden wurde, oder das Ende erreicht ist.
            while currSent <len(sentenceStarts)-1 and sentenceStarts[currSent + 1] <= start:
                currSent += 1
                if(currSent < len(sentenceStarts)-1):
                    currSentLength = sentenceStarts[currSent + 1] - sentenceStarts[currSent]
                else:
                    currSentLength = len(doc) - sentenceStarts[currSent]
            #print(currSentLength)
            #Den Satz in die Liste der gefundenen Sätze des Patterns eintragen, aber nur jeweils einmal
            if(currSent not in sentences[self.nlp.vocab.strings[match_id]]):
                #Mindest Satzlänge beachten
                if(currSentLength >= self.min_length):
                    sentences[self.nlp.vocab.strings[match_id]].append(currSent)
        #print(sentences)

        for key, value in sentences.items():
            if(len(value)>0):
                gp.add_pattern(int(key),value.copy())
        return gp
    

class matcher_localizer:
    #def __init__ (self, *args, **kwargs):
        
        
    def init_matcher(self,spacy_nlp, pattern_terms):
        self.nlp = spacy_nlp
        self.pattern_terms = pattern_terms
        self.matcher = PhraseMatcher(self.nlp.vocab, attr="LOWER")
        #alle Patterns in den Matcher laden
        for patternID, terms in pattern_terms.items():
            patterns = [self.nlp.make_doc(text) for text in terms]
            self.matcher.add(str(patternID),None, *patterns)
        
    def read_nlp_processed_game_to_game_patterns(self,nlp_processed_game):
        gp = game_patterns()
        gp.set_ID(nlp_processed_game.get_ID())
        doc=nlp_processed_game.get_spacy_doc()
        #Alle Satzanfänge ermitteln
        sentenceStarts = []
        sentenceTexts = []
        for sent in doc.sents:
            sentenceStarts.append(sent.start)
            sentenceTexts.append(sent)
        gp.set_sentences
        sentences = {}
        for key, value in self.pattern_terms.items():
            sentences[str(key)] = []
        matches = self.matcher(doc)
        currSent = 0
        #Alle Sätze durchlaufen und Prüfen, ob das aktuelle Match in diesem Satz vorkommt
        #Dabei wird ausgenutzt, dass die Matches sortiert zurück gegeben werden.
        #So müssen die Sätze nur ein einziges mal durchlaufen werden.
        for match_id, start, end in matches:
            #print("ID: "+ nlp.vocab.strings[match_id] + " start: " + str(start) + " pattern found: " + doc[start:end].text)
            
            #Eins weiter bis der Satz mit dem Match gefunden wurde, oder das Ende erreicht ist.
            while currSent <len(sentenceStarts)-1 and sentenceStarts[currSent + 1] < start:
                currSent += 1
            #Den Satz in die Liste der gefundenen Sätze des Patterns eintragen, aber nur jeweils einmal
            if(currSent not in sentences[self.nlp.vocab.strings[match_id]]):
                sentences[self.nlp.vocab.strings[match_id]].append(currSent)
        #print(sentences)

        gp.set_sentence_count(len(sentenceStarts))
        for key, value in sentences.items():
            if(len(value)>0):
                gp.add_pattern(int(key),value.copy())
        return gp
    